In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
os.chdir("drive/My Drive/Colab Notebooks")

In [0]:
import numpy as np
import matplotlib.pyplot as plt
# for creating validation set
from sklearn.model_selection import train_test_split
# for evaluating the model
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import TensorDataset, DataLoader
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout, ELU
from torch.optim import Adam, SGD

In [0]:
X_test = np.load("X_test.npy")
y_test = np.load("y_test.npy")
person_train_valid = np.load("person_train_valid.npy")
X_train_valid = np.load("X_train_valid.npy")
y_train_valid = np.load("y_train_valid.npy")
person_test = np.load("person_test.npy")

In [0]:
print ('Training/Valid data shape: {}'.format(X_train_valid.shape))
print ('Test data shape: {}'.format(X_test.shape))
print ('Training/Valid target shape: {}'.format(y_train_valid.shape))
print ('Test target shape: {}'.format(y_test.shape))
print ('Person train/valid shape: {}'.format(person_train_valid.shape))
print ('Person test shape: {}'.format(person_test.shape))

Training/Valid data shape: (2115, 22, 1000)
Test data shape: (443, 22, 1000)
Training/Valid target shape: (2115,)
Test target shape: (443,)
Person train/valid shape: (2115, 1)
Person test shape: (443, 1)


In [0]:
# change dimensions
# X_train_valid = X_train_valid[:, np.newaxis, :, :]
# X_test = X_test[:, np.newaxis, :, :]
# # create labels 
# y_train_valid = y_train_valid - 769
# y_test = y_test -769

# split training and validation
X_train, X_val, y_train, y_val = train_test_split(X_train_valid, y_train_valid, test_size = 0.1)
data = []
(X_train.shape, y_train.shape), (X_val.shape, y_val.shape)

(((1903, 22, 1000), (1903,)), ((212, 22, 1000), (212,)))

In [0]:
# transform to torch tensor
X_train = torch.from_numpy(X_train)
y_train = torch.from_numpy(y_train)

X_val = torch.from_numpy(X_val)
y_val = torch.from_numpy(y_val)


In [0]:
# process the data in batches for training
batch_size = 100
train_data = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size, drop_last=True)

In [0]:
class Net(Module):
    def __init__(self):
        super(Net, self).__init__()

        self.cnn_layers = Sequential(
            # first conv
            Conv2d(in_channels = 1, out_channels = 16, kernel_size = (1,10)),
            BatchNorm2d(16),
            ELU(inplace=True),
            MaxPool2d(kernel_size=(1,7), stride=(1,4)),
            # output 16*22*247

            # second conv
            Conv2d(in_channels = 16, out_channels = 32, kernel_size = (1,10)),
            BatchNorm2d(32),
            ELU(inplace=True),
            MaxPool2d(kernel_size=(1,6), stride=(1,4)),
            # output 32*22*59

            # third conv
            Conv2d(in_channels = 32, out_channels = 64, kernel_size = (1,10)),
            BatchNorm2d(64),
            ELU(inplace=True),
            MaxPool2d(kernel_size=(1,6), stride=(1,4)),
            # output 64*22*12

            # fourth conv
            Conv2d(in_channels = 64, out_channels = 128, kernel_size = (21,1)),
            BatchNorm2d(128),
            ELU(inplace=True),
            MaxPool2d(kernel_size=(1,4), stride=(1,4)),
            # output 128*2*3
        )

        self.linear_layers = Sequential(
            Linear(128*2*3,4)
        ) 
    
    def forward(self, x):
        x = self.cnn_layers(x)
        x = x.view(x.size(0), -1)
        x = self.linear_layers(x)
        return x
        

In [0]:
# defining the model
model = Net()
model = model.float()
# defining the optimizer
optimizer = Adam(model.parameters(), lr=0.07)
# defining the loss function
criterion = CrossEntropyLoss()
# checking if GPU is available
if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()

print(model)

Net(
  (cnn_layers): Sequential(
    (0): Conv2d(1, 16, kernel_size=(1, 10), stride=(1, 1))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ELU(alpha=1.0, inplace=True)
    (3): MaxPool2d(kernel_size=(1, 7), stride=(1, 4), padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(16, 32, kernel_size=(1, 10), stride=(1, 1))
    (5): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ELU(alpha=1.0, inplace=True)
    (7): MaxPool2d(kernel_size=(1, 6), stride=(1, 4), padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(32, 64, kernel_size=(1, 10), stride=(1, 1))
    (9): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ELU(alpha=1.0, inplace=True)
    (11): MaxPool2d(kernel_size=(1, 6), stride=(1, 4), padding=0, dilation=1, ceil_mode=False)
    (12): Conv2d(64, 128, kernel_size=(21, 1), stride=(1, 1))
    (13): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=Tr

In [0]:
torch.cuda.is_available()

True

In [0]:
if torch.cuda.is_available():
    X_train = X_train.cuda()
    y_train = y_train.cuda()
    X_val = X_val.cuda()
    y_val = y_val.cuda()

##### hyperparameters#########
# defining the optimizer
optimizer = Adam(model.parameters(), lr=1e-3)
n_epochs = 10
batch_size = 100
n_batches = len(y_train)//batch_size

for epoch in range(n_epochs):
    running_loss = 0
    print_every = n_batches // 10
    #start_time = time.time()
    total_train_loss = 0
    for i,data in enumerate(train_loader, 0):
        # extract data in this batch
        inputs, labels = data

        if torch.cuda.is_available():
            inputs = inputs.cuda()
            labels = labels.cuda()
  
        #Set the parameter gradients to zero
        optimizer.zero_grad()
        
        #Forward pass, backward pass, optimize
        outputs = model(inputs.float())
        loss_train = criterion(outputs, labels.long())
        loss_train.backward()
        optimizer.step()

        #Print statistics
        running_loss += loss_train
        total_train_loss += loss_train
        
        #Print every 10th batch of an epoch
        if (i + 1) % (print_every + 1) == 0:
            print("Epoch {}, {:d}% \t train_loss: {:.2f} ".format(
                    epoch+1, int(100 * (i+1) / n_batches), running_loss / print_every))
            #Reset running loss and time
            running_loss = 0.0
        
    #At the end of the epoch, do a pass on the validation set
    output_val = model(X_val.cuda().float())
    loss_val = criterion(output_val, y_val.long())

    print("Validation loss = {:.2f}".format(loss_val))


Epoch 1, 10% 	 train_loss: 0.11 
Epoch 1, 21% 	 train_loss: 0.12 
Epoch 1, 31% 	 train_loss: 0.15 
Epoch 1, 42% 	 train_loss: 0.15 
Epoch 1, 52% 	 train_loss: 0.19 
Epoch 1, 63% 	 train_loss: 0.20 
Epoch 1, 73% 	 train_loss: 0.20 
Epoch 1, 84% 	 train_loss: 0.20 
Epoch 1, 94% 	 train_loss: 0.19 
Validation loss = 1.34
Epoch 2, 10% 	 train_loss: 0.08 
Epoch 2, 21% 	 train_loss: 0.11 
Epoch 2, 31% 	 train_loss: 0.13 
Epoch 2, 42% 	 train_loss: 0.11 
Epoch 2, 52% 	 train_loss: 0.15 
Epoch 2, 63% 	 train_loss: 0.12 
Epoch 2, 73% 	 train_loss: 0.12 
Epoch 2, 84% 	 train_loss: 0.12 
Epoch 2, 94% 	 train_loss: 0.12 
Validation loss = 1.64
Epoch 3, 10% 	 train_loss: 0.08 
Epoch 3, 21% 	 train_loss: 0.07 
Epoch 3, 31% 	 train_loss: 0.08 
Epoch 3, 42% 	 train_loss: 0.07 
Epoch 3, 52% 	 train_loss: 0.07 
Epoch 3, 63% 	 train_loss: 0.09 
Epoch 3, 73% 	 train_loss: 0.05 
Epoch 3, 84% 	 train_loss: 0.06 
Epoch 3, 94% 	 train_loss: 0.07 
Validation loss = 1.62
Epoch 4, 10% 	 train_loss: 0.04 
Epoch 4

In [0]:
## training accuracy
with torch.no_grad():
    output = model(X_train.cuda().float())
    
softmax = torch.exp(output).cpu()
prob = list(softmax.numpy())
predictions = np.argmax(prob, axis=1)

# accuracy on training set
accuracy_score(y_train.cpu(), predictions)

1.0

In [0]:
## validation accuracy
with torch.no_grad():
    output = model(X_val.cuda().float())
softmax = torch.exp(output).cpu()
prob = list(softmax.numpy())
predictions = np.argmax(prob, axis=1)

# accuracy on validation set
accuracy_score(y_val.cpu(), predictions)

0.49056603773584906

The naive CNN above seems to overfit the data. Next let's try GRU mentioned in https://arxiv.org/pdf/1802.00308.pdf


In [0]:
class GRUNet(nn.Module):
    def __init__(self):
        super(GRUNet, self).__init__()
        self.gru = nn.GRU(input_size =22, hidden_size = 32, num_layers = 4, 
                          batch_first=True, dropout=0.2)
        self.FC = Linear(32, 4)
        self.relu = ReLU(inplace=True)
    
    def forward(self, x, h):
        out, h = self.gru(x, h)
        out = self.fc(self.relu(out[:,-1]))
        return out, h




In [0]:
# defining the model
model = GRUNet()
# model = model.float()
# # defining the optimizer
# optimizer = Adam(model.parameters(), lr=0.07)
# # defining the loss function
# criterion = CrossEntropyLoss()
# # checking if GPU is available
# if torch.cuda.is_available():
#     model = model.cuda()
#     criterion = criterion.cuda()

print(model)

GRUNet(
  (gru): GRU(22, 32, num_layers=4, batch_first=True, dropout=0.2)
  (FC): Linear(in_features=32, out_features=4, bias=True)
  (relu): ReLU(inplace=True)
)


In [0]:
optimizer = Adam(model.parameters(), lr=1e-3)
n_epochs = 10
#batch_size = 100
n_batches = len(y_train)//batch_size

for epoch in range(n_epochs):
    running_loss = 0
    print_every = n_batches // 10
    #start_time = time.time()
    total_train_loss = 0
    for i,data in enumerate(train_loader, 0):
        # extract data in this batch
        inputs, labels = data

        if torch.cuda.is_available():
            inputs = inputs.cuda()
            labels = labels.cuda()
  
        #Set the parameter gradients to zero
        optimizer.zero_grad()
        
        #Forward pass, backward pass, optimize
        outputs = model(inputs.float())
        loss_train = criterion(outputs, labels.long())
        loss_train.backward()
        optimizer.step()

        #Print statistics
        running_loss += loss_train
        total_train_loss += loss_train
        
        #Print every 10th batch of an epoch
        if (i + 1) % (print_every + 1) == 0:
            print("Epoch {}, {:d}% \t train_loss: {:.2f} ".format(
                    epoch+1, int(100 * (i+1) / n_batches), running_loss / print_every))
            #Reset running loss and time
            running_loss = 0.0
        
    #At the end of the epoch, do a pass on the validation set
    output_val = model(X_val.cuda().float())
    loss_val = criterion(output_val, y_val.long())

    print("Validation loss = {:.2f}".format(loss_val))

TypeError: ignored